## MuseScore Scraper

Tool: https://github.com/frankye8998/MusicalMusic  

[Reference 2](https://github.com/codeandproduce/music_research_dataset_midi/blob/9a5afe86f1f2d3fdbfa24de99122140e7a7cd60c/MuseScore/MuseScore%20Scraping%20Main.ipynb)

In [1]:
from lxml import html
import urllib
import requests
import shutil

In [2]:
from random import randint
from time import sleep

In [3]:
import json
from pathlib import Path
from tqdm import tqdm
import time

In [4]:
# links = []
# url = "https://musescore.com/hub/piano/solo-piano?sort=view_count"
# # url = "https://musescore.com/hub/piano/piano-duet?sort=view_count"
# page = requests.get(url)
# tree = html.fromstring(page.content)
# print(tree.xpath("//title/text()"))

In [5]:
# articles = tree.xpath("//article[@role='article']")

In [6]:
def get_data(a):
    if a.find("h2//a[@rel='bookmark']") is None:
        a = a.find("div[@class='col-right']")
    score = a.find("h2//a[@rel='bookmark']")
    link = score.attrib['href']
    title = score.text_content().strip()
    author = a.find("div[@class='user']//a").text_content()
    metadata = a.find("div[@class='meta']").text_content().split('•\n')
    parts, pages, duration, time, views = [m.strip() for m in metadata]
    return {
        'link': "https://musescore.com" + link,
        'score_id': Path(link).name,
        'title': title,
        'author': author,
        'parts': parts,
        'pages': pages,
        'duration': duration,
        'views': views
    }

## But that's actually just page one and there are pages 2,3,4,5... 100!
### So let's make a more general format:

In [7]:
# url_default = "https://musescore.com/hub/video_games/movie?sort=view_count&page=" # + the page number
# url_default = 'https://musescore.com/hub/piano/easy-piano?sort=view_count&page='
# url_default = "https://musescore.com/hub/piano/solo-piano?sort=view_count&page=" # + the page number
# url_default = "https://musescore.com/hub/piano/voice-piano?sort=view_count&page=" # + the page number
# url_default = "https://musescore.com/hub/piano?sort=view_count&page=" # + the page number
url_default = "https://musescore.com/groups/piano/sheetmusic?page="

In [8]:
page = requests.get(url_default+str(1)) # https://musescore.com/hub/piano?page=1,2,3,4,5,...,10
tree = html.fromstring(page.content)
articles = tree.xpath("//article[@role='article']")


In [9]:
json_file = Path('musescore_groups.json')

In [9]:
# json_file = Path('song_map.json')
# if json_file.exists():
#     with open(json_file, 'r') as fp:
#         links = json.load(fp)
        
# links = links.values()

In [11]:
if json_file.exists():
    with open(json_file, 'r') as fp:
        links = json.load(fp)
        

In [12]:
def get_number(num_str):
    num_str = num_str.replace(',','').split(' ')[0]
    return int(num_str)

In [13]:
def is_popular_solo(d):
    try:
        views = get_number(d['views'])
        parts = get_number(d['parts'])
    except:
        return False
    return views > 100 and parts == 1

In [14]:
if not json_file.exists():
    links = []
    for page in tqdm(range(1,403), total=403):
        page = requests.get(url_default+str(page)) # https://musescore.com/hub/piano?page=1,2,3,4,5,...,10
        tree = html.fromstring(page.content)

        articles = tree.xpath("//article[@role='article']")
        metadata = [get_data(a) for a in articles]
        metadata = [d for d in metadata if is_popular_solo(d)]
        links.extend(metadata)

#         sleep(randint(1,4))
    with open(json_file, 'w') as fp:
        json.dump(links, fp)
print(links[:100])

[{'link': 'https://musescore.com/user/30284641/scores/5462906', 'score_id': '5462906', 'title': 'The Best Inspiration (50 Follower Special)', 'author': 'Bearsim', 'parts': '1 part', 'pages': '3 pages', 'duration': '02:59', 'views': '431 views'}, {'link': 'https://musescore.com/user/1089721/scores/5521557', 'score_id': '5521557', 'title': 'Sonate in G-moll "Ungestüm" (Impetuous)', 'author': 'Clemenzart ', 'parts': '1 part', 'pages': '8 pages', 'duration': '06:21', 'views': '216 views'}, {'link': 'https://musescore.com/user/1914361/scores/5391567', 'score_id': '5391567', 'title': 'Ice Caves - Original Piano Composition', 'author': 'AnonymousAlchemist', 'parts': '1 part', 'pages': '5 pages', 'duration': '03:53', 'views': '218 views'}, {'link': 'https://musescore.com/user/15026136/scores/5470220', 'score_id': '5470220', 'title': 'Background Music', 'author': 'CountryHomes', 'parts': '1 part', 'pages': '2 pages', 'duration': '02:05', 'views': '471 views'}, {'link': 'https://musescore.com/us

# Scrape links found

### Musical Music Lib

In [10]:

class MuseScoreException(Exception):
    pass

class InvalidFileExtension(MuseScoreException):
    pass

class InvalidScoreID(MuseScoreException):
    pass

class InvalidCredentials(MuseScoreException):
    pass

class InvalidSearchSort(MuseScoreException):
    pass

In [11]:
# import requests

# def urlretrieve(url: str, fn: Path, timeout=10):
#     with fn.open('wb') as f:
#         f.write(requests.get(url, allow_redirects=True, timeout=timeout).content)

In [12]:
import urllib
import urllib.request


import bs4
import requests

class MusicalMusic:
    """Musescore actions requiring an account."""

    def __init__(self, username, password):
        self.username = username
        url = "https://musescore.com/user/login"
        r = requests.get(url)
        soup = bs4.BeautifulSoup(r.text, "html.parser")
        csrf = soup.find("meta", {"name": "csrf-token"})["content"]
        url = "https://musescore.com/user/auth/login/process"
        cookies = {
            "mu_browser_uni": r.cookies['mu_browser_uni'],
            "_csrf": r.cookies["_csrf"]
        }
        data = {
          "username": username,
          "password": password,
          "_csrf": csrf,
          "op": "Log in"
        }
        try:
            mu_user = requests.post(url,
                                    data=data,
                                    cookies=cookies,
                                    allow_redirects=False,
                                    ).cookies["mu_user_new"]
        except KeyError as e:
            raise InvalidCredentials(
                "Please check your username and password!") from e

        mu_browser_uni = r.cookies['mu_browser_uni']

        self.mu_browser_uni = mu_browser_uni
        self.mu_user = mu_user

    def retrieve(self, id, format="pdf"):
        """Retrieves Musescore data in bytes"""

        if format not in ["mp3", "pdf", "mid", "mxl", "mscz"]:
            raise InvalidFileExtension("Must be mp3, pdf, mid, mxl, or mscz.")
        newlink = f"https://musescore.com/score/{id}/download/{format}"
        cookies = {"mu_browser_uni": self.mu_browser_uni,
                   "mu_user_new": self.mu_user}
        bytes = requests.get(newlink, cookies=cookies, verify=False)
        if bytes.status_code != 200:
            raise InvalidScoreID(str(bytes.status_code))
        return bytes.content

    def download(self, id, filename, format="mp3", proxy=None, timeout=10):
        if format not in ["mp3", "pdf", "mid", "mxl", "mscz"]:
            raise InvalidFileExtension("Must be mp3, pdf, mid, mxl, or mscz.")
        newlink = f"https://musescore.com/score/{id}/download/{format}"
        if proxy:
            proxy_support = urllib.request.ProxyHandler({'https' : proxy})
            opener = urllib.request.build_opener(proxy_support)
        else: opener = urllib.request.build_opener()
            
        cookieString = f"mu_browser_uni={self.mu_browser_uni};" \
                        f"mu_user_new={self.mu_user}"
        opener.addheaders = [("cookie", cookieString)]
        urllib.request.install_opener(opener)
        try:
            urllib.request.urlretrieve(newlink, filename)
        except urllib.error.HTTPError as e:
            raise e


### Trying proxy

https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

In [13]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

ua = UserAgent() # From here we generate a random user agent
proxies = [] # Will contain proxies [ip, port]

In [14]:
# Main function
  # Retrieve latest proxies
proxies_req = Request('https://www.sslproxies.org/')
proxies_req.add_header('User-Agent', ua.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')

soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')

# Save proxies in the array
for row in proxies_table.tbody.find_all('tr'):
    proxies.append({
      'ip':   row.find_all('td')[0].string,
      'port': row.find_all('td')[1].string
    })

### More proxies

https://github.com/constverum/ProxyBroker

In [21]:
import asyncio
from proxybroker import Broker

more_proxies = []

async def show(proxy_queue):
    while True:
        proxy = await proxy_queue.get()
        if proxy is None: break
        print('Found proxy: %s' % proxy)
        more_proxies.append({ 'ip': proxy.host, 'port': proxy.port })

proxy_queue = asyncio.Queue()
broker = Broker(proxy_queue)
countries = ['US', 'CA']
tasks = await asyncio.gather(
    broker.find(types=['HTTPS'], countries=countries, max_resp_time=4, limit=100),
    show(proxy_queue))

Found proxy: <Proxy US 0.20s [HTTPS] 165.22.6.247:3128>
Found proxy: <Proxy CA 0.21s [HTTPS] 142.93.63.115:8080>
Found proxy: <Proxy US 0.22s [HTTPS] 216.189.159.74:80>
Found proxy: <Proxy US 0.22s [HTTPS] 138.197.208.221:8080>
Found proxy: <Proxy US 0.32s [HTTPS] 3.87.85.38:3128>
Found proxy: <Proxy CA 0.33s [HTTPS] 138.197.145.16:3128>
Found proxy: <Proxy US 0.55s [HTTPS] 134.209.230.82:8080>
Found proxy: <Proxy US 0.55s [HTTPS] 151.106.10.62:8080>
Found proxy: <Proxy US 0.14s [HTTPS] 35.245.208.185:3128>
Found proxy: <Proxy US 0.15s [HTTPS] 104.248.236.12:8080>
Found proxy: <Proxy US 0.16s [HTTPS] 184.105.143.66:3128>
Found proxy: <Proxy US 0.18s [HTTPS] 35.193.0.225:80>
Found proxy: <Proxy US 0.18s [HTTPS] 162.243.102.207:3128>
Found proxy: <Proxy US 0.18s [HTTPS] 3.17.175.232:3128>
Found proxy: <Proxy US 0.62s [HTTPS] 35.245.153.11:3128>
Found proxy: <Proxy US 0.19s [HTTPS] 134.209.119.225:8080>
Found proxy: <Proxy US 0.19s [HTTPS] 134.209.13.16:8080>
Found proxy: <Proxy US 0.19s 

### Random proxy

In [22]:
# all_proxies = proxies + more_proxies
all_proxies = more_proxies
# Retrieve a random index proxy (we need the index to delete it if not working)
def random_proxy():
    return random.randint(0, len(all_proxies) - 1)

proxy = None
# # Choose a random proxy
# proxy_index = random_proxy()
# proxy = all_proxies[proxy_index]
# proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url

## Actual scraping

In [15]:
invalid_ids = []

In [16]:
deleted_proxies = []

In [17]:
with open('accounts.json', 'r') as fp:
    accounts = json.load(fp)

instances = [MusicalMusic(username, password) for (username, password) in accounts]
instance = None

In [18]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
    if idx % 10 == 0 or instance is None:
        instance_index = random.randint(0, len(instances) - 1)
        instance = instances[instance_index]
    try:
#         print('Downloading score id:', score_id)
        instance.download(score_id, out_file, format='mxl')
    except Exception as e:
        print('Could not download id:', score_id, e)
        instance = None
    

 30%|███       | 4097/13581 [00:41<11:45, 13.45it/s]  

Could not download id: 2693876 HTTP Error 403: Forbidden


 33%|███▎      | 4414/13581 [00:42<08:08, 18.75it/s]

Could not download id: 3970536 HTTP Error 403: Forbidden


 35%|███▌      | 4817/13581 [00:42<05:30, 26.54it/s]

...

 93%|█████████▎| 12574/13581 [01:36<04:39,  3.60it/s]

Could not download id: 59325 HTTP Error 404: Not Found


 93%|█████████▎| 12634/13581 [01:36<03:05,  5.10it/s]

Could not download id: 49110 HTTP Error 404: Not Found


100%|██████████| 13581/13581 [02:13<00:00, 101.47it/s]


In [36]:
len(all_proxies)

57

In [37]:
import time

In [38]:
for idx,link in enumerate(tqdm(links, total=len(links))):
    score_id = link['score_id']
    out_file = f"data/{score_id}.mxl"
    if Path(out_file).exists() or score_id in invalid_ids: continue
    # Every 10 requests, generate a new proxy
#     if idx % 10 == 0 or proxy is None or instance is None:
    proxy_index = random.randint(0, len(all_proxies) - 1)
    proxy = all_proxies[proxy_index]
    proxy_url = proxy['ip'] + ':' + str(proxy['port']); proxy_url
    instance_index = random.randint(0, len(instances) - 1)
    instance = instances[instance_index]
    try:
        print('Downloading score id:', score_id)
        start = time.time()
        instance.download(score_id, out_file, format='mxl', proxy=proxy_url)
        end = time.time()
        if (end - start) > 10: raise Exception('Took too long to download')
    except Exception as e:
        print('Could not download id:', score_id)
        print('Error:', e)
        invalid_ids.append(score_id)
        deleted_proxies.append(all_proxies[proxy_index])
        del all_proxies[proxy_index]
        print('Proxy ' + proxy['ip'] + ':' + str(proxy['port']) + ' deleted.')
        print(instance.username)
        proxy = None
        instance = None
    






  0%|          | 0/13582 [00:00<?, ?it/s]






 24%|██▍       | 3290/13582 [00:03<00:12, 847.01it/s]

Could not download id: 5100702
Error: Took too long to download
Proxy 157.230.90.113:8080 deleted.
tulebo







 24%|██▍       | 3290/13582 [00:21<00:12, 847.01it/s]




 24%|██▍       | 3294/13582 [00:23<4:16:18,  1.49s/it]






 24%|██▍       | 3295/13582 [00:26<5:10:26,  1.81s/it]

...






 26%|██▌       | 3528/13582 [10:17<6:02:55,  2.17s/it]






 26%|██▌       | 3529/13582 [10:19<5:27:13,  1.95s/it]






 26%|██▌       | 3530/13582 [10:21<5:12:23,  1.86s/it]






 26%|██▌       | 3531/13582 [10:22<5:03:28,  1.81s/it]

KeyboardInterrupt: 

In [34]:
del all_proxies[proxy_index]
proxy = None